In [1]:
from new import *
from IPython.core.display import display
%reload_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', 30)
ROOT = Path.cwd()
DATA_ROOT = ROOT.parent.parent / 'Datasets' / '2ndData'


In [33]:
def calculate_max_angle(fromX,fromY,fromZ,toX,toY,toZ,r):
    length = np.sqrt((toX-fromX)**2 + (toY-fromY)**2 + (toZ-fromZ)**2)
    angle_in_rad=np.arcsin(r/length)

    return np.rad2deg(angle_in_rad)

def calculate_current_angle(headX,headY,headZ,fromX,fromY,fromZ,toX,toY,toZ):
    import vg
    vector1 = np.array([headX,headY,headZ])
    vector2 = np.array([toX-fromX,toY-fromY,toZ-fromZ])
    return vg.angle(vector1,vector2)

def add_columns(holo):
    rs = []
    thetas = []
    phis = []
    for index, row in holo.iterrows():
        x = row['TargetPositionX'] - row['HeadPositionX']
        y = row['TargetPositionY'] - row['HeadPositionY']
        z = row['TargetPositionZ'] - row['HeadPositionZ']
        [r, theta, phi] = Analysing_functions.asSpherical([x, z, y])
        rs.append(r)
        thetas.append(90 - theta)
        phis.append(90 - phi)

    holo['R'] = rs
    holo['Theta'] = thetas
    holo['Phi'] = phis
    holo['TargetVertical'] = holo.HeadRotationX + holo.Theta
    holo['TargetHorizontal'] = holo.HeadRotationY - holo.Phi
    for col in ['HeadForwardX', 'HeadForwardY', 'HeadForwardZ']:
        holo[str(col) + '_next'] = holo[col].shift(1)
    holo['time_interval'] = holo.Timestamp.diff()
    holo['angle_speed'] = holo.apply(lambda x: angle_velocity(x.HeadForwardX, x.HeadForwardY, x.HeadForwardZ,
                                                                        x.HeadForwardX_next, x.HeadForwardY_next,
                                                                        x.HeadForwardZ_next,
                                                                        x.time_interval), axis=1)
    return holo

def angle_velocity(x1, y1, z1, x2, y2, z2, _time):
    import vg
    # if type(_head_forward2) is not dict: return None
    vector1 = np.array([x1, y1, z1])
    vector2 = np.array([x2, y2, z2])
    return vg.angle(vector1, vector2) / _time

subjects = subjects = range(201, 212)
poss = ['W','S']
envs = ['U','W']
targets=range(8)
# blocks=range(5)
blocks = [1,2,3,4]
# subjects= [201]
# poss=['W']
# envs=['W']
# targets=[0]
# blocks=[0]
output=pd.DataFrame()
# TODO : summary save for each subject
for subject in subjects:
    output=pd.DataFrame()
    [imu_file_list, eye_file_list, hololens_file_list] = FileHandling.get_one_subject_files(subject, refined=True)
    for target, env, pos, block in itertools.product(targets, envs, poss, blocks):
        current_info = [target, env, pos, block]
        try:
            print('analysing',subject,current_info)

            holo = FileHandling.file_as_pandas(FileHandling.get_file_by_info(hololens_file_list, current_info))
            holo.Timestamp  = holo.Timestamp - holo.Timestamp[0]
            holo = add_columns(holo)
            if holo[holo.TargetEntered=='target_'+str(target)].shape[0]<1 :
                print("Fail in trial")
                continue
            start_index = holo[holo.TargetEntered=='target_'+str(target)].index[0]

            # Watch after first target-on


            holo_after_target_on = holo[holo.index > start_index]
            start_time = holo_after_target_on.Timestamp.values[0]
            # --- Behavioural statistics ---
            # Accuracy : mean distance
            accuracy_vertical =-holo_after_target_on.Theta.mean() - holo_after_target_on.HeadRotationX.mean()
            accuracy_horizontal = holo_after_target_on.Phi.mean() - holo_after_target_on.HeadRotationY.mean()
            # Precision : standard deviation
            precision_vertical = (holo_after_target_on.TargetVertical).std()
            precision_horizontal = (holo_after_target_on.TargetHorizontal).std()
            # print('accuracy:',accuracy_vertical,accuracy_horizontal)
            # print('precision:',precision_vertical,precision_horizontal)

            # --- Target-wise ---
            if env=='W':
                r=0.25
            elif env=='S':
                r=0.25/5
            else:
                print("wrong env");continue
            holo['max_angle_distance'] = holo.apply(lambda x:calculate_max_angle(x['HeadPositionX'],x['HeadPositionY'],x['HeadPositionZ'],
                                                                                 x['TargetPositionX'],x['TargetPositionY'],x['TargetPositionZ'],r),axis=1)
            holo['current_angle_distance'] = holo.apply(lambda x: calculate_current_angle(x['HeadForwardX'],x['HeadForwardY'],x['HeadForwardZ'],x['HeadPositionX'],x['HeadPositionY'],x['HeadPositionZ'],
                                                                                 x['TargetPositionX'],x['TargetPositionY'],x['TargetPositionZ']),axis=1)
            holo['targetOn'] = holo.apply(lambda x: x['current_angle_distance'] < x['max_angle_distance'], axis=1)
            holo['targetOn_prev'] = holo.targetOn.shift(1)
            dwells=list()
            temp_dwell=list()
            for index,row in holo.iterrows():
                if row['targetOn'] == True and row['targetOn_prev']==False:
                    temp_dwell=list()
                    temp_dwell.append(row)
                elif row['targetOn'] == True and row['targetOn_prev']==True:
                    temp_dwell.append(row)
                elif row['targetOn'] == False and row['targetOn_prev']==True:
                    if temp_dwell:
                        dwells.append(pd.DataFrame(temp_dwell))
                    temp_dwell=list()
                if index==holo.index[-1]:
                    if temp_dwell :
                        dwells.append(pd.DataFrame(temp_dwell))


            target_on_count = len(dwells)
            if target_on_count > 0:
                longest_target_on = max([x.Timestamp.values[-1]-x.Timestamp.values[0] for x in dwells])
            else:
                print('No target-on')
                continue
                longest_target_on=-1
            mean_speeds=list()
            target_in_speeds=list()
            target_out_speeds=list()
            last_100ms_speeds=list()
            for dwell in dwells:
                mean_speeds.append(dwell.angle_speed.mean())
                target_in_speeds.append(dwell.angle_speed.values[0])
                target_out_speeds.append(dwell.angle_speed.values[-1])
                dwell_last100 = dwell[dwell.Timestamp > dwell.Timestamp.values[-1]-100/1000]
                last_100ms_speeds.append(dwell_last100.angle_speed.mean())
            thresholds=[100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500]
            for threshold in thresholds:
                dwell_threshold = threshold
                success_dwells=list()
                total_duration=0
                first_dwell_timestamp = -1;first_success_dwell_timestamp=-1
                prior_count=0
                for index, dwell in enumerate(dwells):
                    if index==0: first_dwell_timestamp=dwell.Timestamp.values[0]
                    duration = dwell.Timestamp.values[-1]-dwell.Timestamp.values[0]
                    total_duration += duration
                    if duration >dwell_threshold/1000:
                        if first_success_dwell_timestamp<0:
                            prior_count=index
                            first_success_dwell_timestamp=dwell.Timestamp.values[0]
                        success_dwells.append(dwell)

                success_dwell_count = len(success_dwells)
                result = dict(
                    subject=subject,target=target, env=env, pos=pos, block=block,
                    accuracy_vertical=accuracy_vertical,accuracy_horizontal=accuracy_horizontal,
                    precision_vertical=precision_vertical,precision_horizontal=precision_horizontal,
                    dwell_threshold=dwell_threshold,dwells=dwells,success_dwells=success_dwells,
                    target_on_count=target_on_count,success_dwell_count=success_dwell_count,
                    total_duration = total_duration,mean_duration = total_duration/target_on_count,
                    first_dwell_timestamp = first_dwell_timestamp, first_success_dwell_timestamp=first_success_dwell_timestamp,longest_target_on=longest_target_on,
                    mean_speeds=mean_speeds,target_in_speeds=target,target_out_speeds=target_out_speeds,last_100ms_speeds=last_100ms_speeds,
                    prior_count=prior_count
                )
                output=output.append(result,ignore_index=True)
        except ValueError as err:
            print(err, current_info)
    output.to_pickle("summary"+str(subject-200)+".pkl")
print(output.shape)

analysing 201 [0, 'U', 'W', 1]
wrong env
analysing 201 [0, 'U', 'W', 2]
wrong env
analysing 201 [0, 'U', 'W', 3]
wrong env
analysing 201 [0, 'U', 'W', 4]
wrong env
analysing 201 [0, 'U', 'S', 1]
wrong env
analysing 201 [0, 'U', 'S', 2]
wrong env
analysing 201 [0, 'U', 'S', 3]
wrong env
analysing 201 [0, 'U', 'S', 4]
wrong env
analysing 201 [0, 'W', 'W', 1]
analysing 201 [0, 'W', 'W', 2]
analysing 201 [0, 'W', 'W', 3]
analysing 201 [0, 'W', 'W', 4]
analysing 201 [0, 'W', 'S', 1]
analysing 201 [0, 'W', 'S', 2]
analysing 201 [0, 'W', 'S', 3]
analysing 201 [0, 'W', 'S', 4]
analysing 201 [1, 'U', 'W', 1]
wrong env
analysing 201 [1, 'U', 'W', 2]
wrong env
analysing 201 [1, 'U', 'W', 3]
wrong env
analysing 201 [1, 'U', 'W', 4]
wrong env
analysing 201 [1, 'U', 'S', 1]
wrong env
analysing 201 [1, 'U', 'S', 2]
wrong env
analysing 201 [1, 'U', 'S', 3]
wrong env
analysing 201 [1, 'U', 'S', 4]
wrong env
analysing 201 [1, 'W', 'W', 1]
analysing 201 [1, 'W', 'W', 2]
analysing 201 [1, 'W', 'W', 3]
ana

In [14]:
output.to_pickle("summary.pkl")

In [36]:
summary = pd.read_pickle("summary3.pkl")

In [38]:
behaviour_results=pd.DataFrame()
for subject in subjects:
    summary = pd.read_pickle("summary"+str(subject-200)+".pkl")
    standing = summary[(summary.pos=='S') & (summary.dwell_threshold==1500)]
    walking =  summary[(summary.pos=='W') & (summary.dwell_threshold==1500)]
    acc_H_stand = standing.accuracy_horizontal.abs().mean()
    acc_V_stand = standing.accuracy_vertical.abs().mean()
    acc_H_walk = walking.accuracy_horizontal.abs().mean()
    acc_V_walk = walking.accuracy_vertical.abs().mean()
    pre_H_stand = standing.precision_horizontal.mean()
    pre_V_stand = standing.precision_vertical.mean()
    pre_H_walk = walking.precision_horizontal.mean()
    pre_V_walk = walking.precision_vertical.mean()
    print('accuracy',acc_H_stand,'->',acc_H_walk)
    print('accuracy',acc_V_stand,'->',acc_V_walk)
    print('precision',pre_H_stand,'->',pre_H_walk)
    print('precision',pre_V_stand,'->',pre_V_walk)
    result = dict(
        subject=subject,
        stand_accuracy_horizontal = acc_H_stand,
        stand_accuracy_vertical = acc_V_stand,
        walk_accuracy_horizontal = acc_H_walk,
        walk_accuracy_vertical = acc_V_walk,
        stand_precision_horizontal = pre_H_stand,
        stand_precision_vertical = pre_V_stand,
        walk_precision_horizontal = pre_H_walk,
        walk_precision_vertical = pre_V_walk,
    )
    behaviour_results = behaviour_results.append(result,ignore_index=True)

accuracy 0.21671975697687462 -> 0.3718668046378664
accuracy 0.24754625126936294 -> 0.29653405442552305
precision 0.34684949511938873 -> 1.1351656114414914
precision 0.4279700597261817 -> 1.1811047204426055
accuracy 0.21168598734024577 -> 0.2777855155192628
accuracy 0.25415967414250107 -> 0.787518891426928
precision 0.22678109681665376 -> 1.14552926816032
precision 0.3052993769483194 -> 1.5386333036191204
accuracy 0.12295042297206527 -> 0.11989336017777671
accuracy 0.20485259771029196 -> 0.15441834718338973
precision 0.2593200744659362 -> 0.7740506664539576
precision 0.32843438204905384 -> 0.8163679254095753
accuracy 0.1332055771661355 -> 0.19907554393220483
accuracy 0.23431281626744985 -> 0.3681776658032358
precision 0.2722008707817874 -> 1.0474429450878109
precision 0.3112500872634455 -> 1.0030770058065834
accuracy 0.20256163635386432 -> 0.32957071701152973
accuracy 0.21580555533252826 -> 0.5169710188956793
precision 0.2103855540197599 -> 0.8541952293001471
precision 0.237729096595505

In [46]:
import plotly.graph_objects as go
cols=["accuracy_H","accuracy_V"]
fig = go.Figure(
    data=[
        go.Bar(name='Standing',error_y=dict(type='data',array=[behaviour_results.stand_accuracy_horizontal.std(),behaviour_results.stand_accuracy_vertical.std()]),x=cols,y=[behaviour_results.stand_accuracy_horizontal.mean(),behaviour_results.stand_accuracy_vertical.mean()]),
        go.Bar(name='Walking',error_y=dict(type='data',array=[behaviour_results.walk_accuracy_horizontal.std(),behaviour_results.walk_accuracy_vertical.std()]),x=cols,y=[behaviour_results.walk_accuracy_horizontal.mean(),behaviour_results.walk_accuracy_vertical.mean()]),
        # go.Bar(x=,y=behaviour_results.stand_accuracy_vertical.mean()),
    ]
)
fig.update_layout(barmode='group')
fig.show()
cols=['precision_H','precision_V']
fig=go.Figure(
    data=[
        go.Bar(name='Standing',error_y=dict(type='data',array=[behaviour_results.stand_precision_horizontal.std(),behaviour_results.stand_precision_vertical.std()]),
               x=cols,y=[behaviour_results.stand_precision_horizontal.mean(),behaviour_results.stand_precision_vertical.mean()]),
        go.Bar(name='Walking',error_y=dict(type='data',array=[behaviour_results.walk_precision_horizontal.std(),behaviour_results.walk_precision_vertical.std()]),
               x=cols,y=[behaviour_results.walk_precision_horizontal.mean(),behaviour_results.walk_precision_vertical.mean()]),
    ]
)
fig.update_layout(barmode='group')
fig.show()

In [2]:
dwell_results=pd.DataFrame()
for subject in subjects:
    summary = pd.read_pickle("summary"+str(subject-200)+".pkl")
    standing = summary[(summary.pos=='S')]
    walking =  summary[(summary.pos=='W')]
    for temp_threshold in [100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500]:
        temp_standing = standing[standing.dwell_threshold==temp_threshold]
        temp_walking = walking[walking.dwell_threshold == temp_threshold]
        temp_standing.target_on_count